VAMOS A TRABAJAR CON EL ARCHIVO CREADO EN EL EJERCICIO 1 "transacciones" en HDFS

In [6]:
#con spark context levanto los n archivos del cluster y lo trato como un solo archivo
#spark context ya lo trae por defecto el notebook trabajando sobre cluster por eso no lo creo.

from pyspark.sql import SparkSession #permite interactuar con Dataframe (datos en memoria)

from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-06-11T22:32:57,752 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<b>Cargo los datos en un dataframe de spark (EN MEMORIA)</b>

In [3]:
#cargamos en un dataframe en memoria el archivo que necesito del cluster
#Si bien guardamos como texto separado por comas en ejercicio 1, podría ser otro el separador.

df = spark.read.csv('output/raw/transacciones', header=False, sep='|')

In [4]:
#este show es como el de Pandas, pero Pandas es Onecore por lo que para millones de registros pandas no sirve.
#Es un dataframe de Spark, por tanto esta distribuido y funciona en cluster y lo vemos en memoria.

df.show()

+--------------------+--------------------+------+----+
|                 _c0|                 _c1|   _c2| _c3|
+--------------------+--------------------+------+----+
|7a2abb43-6cc5-46a...|           SUPERCITO|RETIRO|7264|
|6c8f3d4c-2dab-4bc...|           SUPERCITO|COMPRA|1937|
|6c8f3d4c-2dab-4bc...|           SUPERCITO|COMPRA|2711|
|34220fd5-02d3-48c...|RESTAURANTE EL TR...|RETIRO|8026|
|f4922ec0-9318-40a...|           SUPERCITO|COMPRA|5125|
|f4922ec0-9318-40a...|       ARENA COLISEO|RETIRO|3432|
|b26797f5-6581-419...|RESTAURANTE EL TR...|RETIRO|2132|
|6c8f3d4c-2dab-4bc...|RESTAURANTE EL TR...|COMPRA|2566|
|f4922ec0-9318-40a...|RESTAURANTE EL TR...|RETIRO|5815|
|34220fd5-02d3-48c...|       ARENA COLISEO|COMPRA|1954|
|7a2abb43-6cc5-46a...|           SUPERCITO|COMPRA|5277|
|b26797f5-6581-419...|       ARENA COLISEO|RETIRO|5330|
|7a2abb43-6cc5-46a...|RESTAURANTE EL TR...|RETIRO|4156|
|b26797f5-6581-419...|           SUPERCITO|RETIRO|3737|
|34220fd5-02d3-48c...|           SUPERCITO|COMPR

In [5]:
# imprime el tipo de datos que spark comprende por defecto.
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



<b>En el dataframe anterior, como los datos no tienen un cabezal, nos queda con un cabezal por defecto</b>

In [6]:
# Puedo definir el esquema con Sruct (como dato complejo de Python), acá le cambiamos el último tipo de datos al dato que conozco que es float.
# Le agregamos nombres a las columnas "header" también

schema = StructType([\
    StructField("id_cliente", StringType(), True),\
    StructField("comercio", StringType(), True),\
    StructField("accion", StringType(), True),
    StructField("monto", FloatType(), True)
])

In [7]:
#Yo le indico cual esquema es el que debe usar cuando traiga la data del cluster, en este caso "schema"
df = spark.read.csv('output/raw/transacciones', header=False, sep='|', schema=schema)

In [8]:
df

DataFrame[id_cliente: string, comercio: string, accion: string, monto: float]

In [9]:
df.show()

+--------------------+--------------------+------+------+
|          id_cliente|            comercio|accion| monto|
+--------------------+--------------------+------+------+
|7a2abb43-6cc5-46a...|           SUPERCITO|RETIRO|7264.0|
|6c8f3d4c-2dab-4bc...|           SUPERCITO|COMPRA|1937.0|
|6c8f3d4c-2dab-4bc...|           SUPERCITO|COMPRA|2711.0|
|34220fd5-02d3-48c...|RESTAURANTE EL TR...|RETIRO|8026.0|
|f4922ec0-9318-40a...|           SUPERCITO|COMPRA|5125.0|
|f4922ec0-9318-40a...|       ARENA COLISEO|RETIRO|3432.0|
|b26797f5-6581-419...|RESTAURANTE EL TR...|RETIRO|2132.0|
|6c8f3d4c-2dab-4bc...|RESTAURANTE EL TR...|COMPRA|2566.0|
|f4922ec0-9318-40a...|RESTAURANTE EL TR...|RETIRO|5815.0|
|34220fd5-02d3-48c...|       ARENA COLISEO|COMPRA|1954.0|
|7a2abb43-6cc5-46a...|           SUPERCITO|COMPRA|5277.0|
|b26797f5-6581-419...|       ARENA COLISEO|RETIRO|5330.0|
|7a2abb43-6cc5-46a...|RESTAURANTE EL TR...|RETIRO|4156.0|
|b26797f5-6581-419...|           SUPERCITO|RETIRO|3737.0|
|34220fd5-02d3

In [10]:
df.printSchema()

root
 |-- id_cliente: string (nullable = true)
 |-- comercio: string (nullable = true)
 |-- accion: string (nullable = true)
 |-- monto: float (nullable = true)



In [11]:
#cuantos registros tiene
# corre varios programas java sobre las particiones elvanta memoria, cuenta registros, temrina, lo pasa al padre, él lo junta y devuelve resultado.  

df.count()

1000000

In [12]:
# Casi misma sintaxis de Panda
# No lo pongo en el mismo data frame (DF) sino que uso otro porque los DF son inmutables.
# Si puedo hacer transformaciones para llevarlo a un nuevo DF ("df2")

df2 = df.filter(df.comercio == "SUPERCITO")

#Por ser transformación no demora a diferencia del show que es una acción.

In [15]:
# Mejor correr todas las Transformaciones necesarias antes de Acciones
# Todo esto está en Memoría
df2.show()

+--------------------+---------+------+------+
|          id_cliente| comercio|accion| monto|
+--------------------+---------+------+------+
|7a2abb43-6cc5-46a...|SUPERCITO|RETIRO|7264.0|
|6c8f3d4c-2dab-4bc...|SUPERCITO|COMPRA|1937.0|
|6c8f3d4c-2dab-4bc...|SUPERCITO|COMPRA|2711.0|
|f4922ec0-9318-40a...|SUPERCITO|COMPRA|5125.0|
|7a2abb43-6cc5-46a...|SUPERCITO|COMPRA|5277.0|
|b26797f5-6581-419...|SUPERCITO|RETIRO|3737.0|
|34220fd5-02d3-48c...|SUPERCITO|COMPRA|6142.0|
|b26797f5-6581-419...|SUPERCITO|RETIRO|1566.0|
|34220fd5-02d3-48c...|SUPERCITO|RETIRO|9472.0|
|b26797f5-6581-419...|SUPERCITO|RETIRO|8017.0|
|34220fd5-02d3-48c...|SUPERCITO|COMPRA|5039.0|
|6c8f3d4c-2dab-4bc...|SUPERCITO|COMPRA|1594.0|
|7a2abb43-6cc5-46a...|SUPERCITO|RETIRO|1469.0|
|b26797f5-6581-419...|SUPERCITO|RETIRO|5357.0|
|6c8f3d4c-2dab-4bc...|SUPERCITO|COMPRA|3256.0|
|b26797f5-6581-419...|SUPERCITO|RETIRO|9540.0|
|7a2abb43-6cc5-46a...|SUPERCITO|COMPRA|8336.0|
|f4922ec0-9318-40a...|SUPERCITO|COMPRA|4442.0|
|b26797f5-658

In [16]:
# Quiero que agrupe por Comercio y me muestre el monto.
# como es un show no lo guardo en otro DF, si necesitara exportarlo o alfgo usar un DF

df.groupBy("comercio").sum("monto").show(truncate=False)

+----------------------+-------------+
|comercio              |sum(monto)   |
+----------------------+-------------+
|SUPERCITO             |1.672819868E9|
|RESTAURANTE EL TRABAJO|1.666856211E9|
|ARENA COLISEO         |1.665255751E9|
+----------------------+-------------+



<b>Vamos a crear una tabla sql y trabajar como si fuera sql standard</b>

In [17]:
# Ya no voy a tratar como archivo csv sino como sql (como una tabla) 
# crea en memoria tabla temporal llamada transacciones

df.createOrReplaceTempView("transacciones")

In [18]:
spark.sql("select * from transacciones").show()

+--------------------+--------------------+------+------+
|          id_cliente|            comercio|accion| monto|
+--------------------+--------------------+------+------+
|7a2abb43-6cc5-46a...|           SUPERCITO|RETIRO|7264.0|
|6c8f3d4c-2dab-4bc...|           SUPERCITO|COMPRA|1937.0|
|6c8f3d4c-2dab-4bc...|           SUPERCITO|COMPRA|2711.0|
|34220fd5-02d3-48c...|RESTAURANTE EL TR...|RETIRO|8026.0|
|f4922ec0-9318-40a...|           SUPERCITO|COMPRA|5125.0|
|f4922ec0-9318-40a...|       ARENA COLISEO|RETIRO|3432.0|
|b26797f5-6581-419...|RESTAURANTE EL TR...|RETIRO|2132.0|
|6c8f3d4c-2dab-4bc...|RESTAURANTE EL TR...|COMPRA|2566.0|
|f4922ec0-9318-40a...|RESTAURANTE EL TR...|RETIRO|5815.0|
|34220fd5-02d3-48c...|       ARENA COLISEO|COMPRA|1954.0|
|7a2abb43-6cc5-46a...|           SUPERCITO|COMPRA|5277.0|
|b26797f5-6581-419...|       ARENA COLISEO|RETIRO|5330.0|
|7a2abb43-6cc5-46a...|RESTAURANTE EL TR...|RETIRO|4156.0|
|b26797f5-6581-419...|           SUPERCITO|RETIRO|3737.0|
|34220fd5-02d3

In [19]:
spark.sql("select comercio, sum(monto) as  total from transacciones group by comercio").show()

+--------------------+-------------+
|            comercio|        total|
+--------------------+-------------+
|           SUPERCITO|1.672819868E9|
|RESTAURANTE EL TR...|1.666856211E9|
|       ARENA COLISEO|1.665255751E9|
+--------------------+-------------+



<b>Ejercicio 1<b><p></p>
Para cada comercio obtener y mostrar los máximos montos por acción.

In [20]:
spark.sql("select comercio, accion, max(monto) as montoMax from transacciones group by comercio, accion").show()

+--------------------+------+--------+
|            comercio|accion|montoMax|
+--------------------+------+--------+
|           SUPERCITO|COMPRA| 10000.0|
|           SUPERCITO|RETIRO| 10000.0|
|       ARENA COLISEO|RETIRO| 10000.0|
|RESTAURANTE EL TR...|COMPRA| 10000.0|
|RESTAURANTE EL TR...|RETIRO| 10000.0|
|       ARENA COLISEO|COMPRA| 10000.0|
+--------------------+------+--------+



<b>Ejercicio 2<b><p></p>
Obtener y mostrar el top 10 de cantidad de transacciones por cliente y comercio

In [38]:

spark.sql("select id_cliente, comercio, count(accion) as cantTransacciones from transacciones group by id_cliente, comercio order by cantTransacciones DESC LIMIT 10").show()

+--------------------+--------------------+-----------------+
|          id_cliente|            comercio|cantTransacciones|
+--------------------+--------------------+-----------------+
|6c8f3d4c-2dab-4bc...|           SUPERCITO|            67127|
|7a2abb43-6cc5-46a...|           SUPERCITO|            66899|
|f4922ec0-9318-40a...|           SUPERCITO|            66859|
|7a2abb43-6cc5-46a...|RESTAURANTE EL TR...|            66810|
|34220fd5-02d3-48c...|       ARENA COLISEO|            66770|
|34220fd5-02d3-48c...|           SUPERCITO|            66757|
|b26797f5-6581-419...|       ARENA COLISEO|            66652|
|6c8f3d4c-2dab-4bc...|       ARENA COLISEO|            66624|
|f4922ec0-9318-40a...|       ARENA COLISEO|            66606|
|34220fd5-02d3-48c...|RESTAURANTE EL TR...|            66604|
+--------------------+--------------------+-----------------+



<b>DATAFRAMES DESDE JSON</B>

In [1]:
#Borrar este archivo por si lo tenemos en la Virtual Machine (de otras practicas)
! rm -R data/world_bank*

In [2]:
#Traemos el archivo a nuestra máquina virtual en una carpeta que crea llamada data
! wget https://github.com/bradenrc/sparksql_pot/raw/master/world_bank.zip -P data/

--2024-06-11 22:27:42--  https://github.com/bradenrc/sparksql_pot/raw/master/world_bank.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bradenrc/sparksql_pot/master/world_bank.zip [following]
--2024-06-11 22:27:42--  https://raw.githubusercontent.com/bradenrc/sparksql_pot/master/world_bank.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 446064 (436K) [application/zip]
Saving to: ‘data/world_bank.zip’

world_bank.zip      100%[===================>] 435.61K  --.-KB/s    in 0.002s  

2024-06-11 22:27:42 (174 MB/s) - ‘data/world_bank.zip’ saved [446064/446064]



In [3]:
# Descomprimir el archivo .zip en data 
! unzip data/world_bank.zip -d data/world_bank

Archive:  data/world_bank.zip
  inflating: data/world_bank/world_bank.json  


In [4]:
# Borramos el .zip (seguimos en VM)
! rm data/world_bank.zip

In [7]:
#Leemos el json
world_bank = spark.read.json("file:///home/ort/notebooks/data/world_bank/world_bank.json")

In [8]:
#leer el esquema que comprende spark
world_bank.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- approvalfy: string (nullable = true)
 |-- board_approval_month: string (nullable = true)
 |-- boardapprovaldate: string (nullable = true)
 |-- borrower: string (nullable = true)
 |-- closingdate: string (nullable = true)
 |-- country_namecode: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- countryname: string (nullable = true)
 |-- countryshortname: string (nullable = true)
 |-- docty: string (nullable = true)
 |-- envassesmentcategorycode: string (nullable = true)
 |-- grantamt: long (nullable = true)
 |-- ibrdcommamt: long (nullable = true)
 |-- id: string (nullable = true)
 |-- idacommamt: long (nullable = true)
 |-- impagency: string (nullable = true)
 |-- lendinginstr: string (nullable = true)
 |-- lendinginstrtype: string (nullable = true)
 |-- lendprojectcost: long (nullable = true)
 |-- majorsector_percent: array (nullable = true)
 |    |-- element: struct (containsNu

In [9]:
#El mismo json en formato Tabular
world_bank.show()

2024-06-11T22:34:30,001 WARN [Thread-4] org.apache.spark.sql.catalyst.util.package - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+------------------+--------------------+------------------------+--------+-----------+-------+----------+--------------------+--------------------+----------------+---------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+------+

In [10]:
#creamos la tabla en memoria para poder levantarlo y trabajarlo como sql
world_bank.createOrReplaceTempView("world_bank_projects")

In [11]:
spark.sql('select countryshortname, project_name, totalamt, totalcommamt from world_bank_projects order by countryshortname').show()

+----------------+--------------------+---------+------------+
|countryshortname|        project_name| totalamt|totalcommamt|
+----------------+--------------------+---------+------------+
|     Afghanistan|Afghanistan Agric...|        0|    74730000|
|     Afghanistan|AF: Development P...| 50000000|    50000000|
|     Afghanistan|Afghanistan: Safe...| 12500000|    12500000|
|     Afghanistan|Afghanistan - Sec...| 55000000|    55000000|
|     Afghanistan|Afghanistan: Syst...|100000000|   100000000|
|     Afghanistan|Afghanistan: Nati...|        0|   100000000|
|          Africa|West Africa  Regi...|        0|    10000000|
|          Africa|RCIP4 - Regional ...| 22000000|    22000000|
|          Africa|Building Resilien...|        0|     4630000|
|          Africa|Fighting against ...|        0|     2000000|
|          Africa|Regional Rusumo F...|339900000|   339900000|
|          Africa|Additional Financ...| 60000000|    60000000|
|          Africa|West Africa Regio...| 60000000|    60

In [24]:
projects_by_country = spark.sql('select countryshortname as country, count(project_name) as num_projects, sum(totalamt) as total_amount from world_bank_projects group by countryshortname order by total_amount desc')
projects_by_country.show()


+--------------------+------------+------------+
|             country|num_projects|total_amount|
+--------------------+------------+------------+
|               India|          16|  2595700000|
|              Brazil|           9|  2326200000|
|           Indonesia|          19|  2045500000|
|             Vietnam|          17|  1832400000|
|          Bangladesh|          12|  1566500000|
|               China|          19|  1540000000|
|              Poland|           1|  1307800000|
|              Turkey|           4|  1301000000|
|            Ethiopia|           4|  1245000000|
|             Nigeria|           7|  1220000000|
|              Africa|          11|   987900000|
|            Colombia|           3|   950000000|
|               Kenya|           6|   875000000|
|             Morocco|          12|   793200000|
|            Pakistan|           9|   744000000|
|             Myanmar|           3|   660000000|
|          Mozambique|          11|   657000000|
|            Tanzani

<b>Ejercicio 3<b><p></p>
A partir del dataframe projects_by_country encontrar el monto promedio por proyecto para Uruguay y Bolivia

In [15]:
projects_by_country.createOrReplaceTempView("projects_by_country")


In [17]:
projects_by_country.printSchema()

root
 |-- country: string (nullable = true)
 |-- num_projects: long (nullable = false)
 |-- total_amount: long (nullable = true)



In [21]:
spark.sql('select country, AVG(total_amount/num_projects) as monto_Promedio_Proyecto from projects_by_country where country = "Bolivia" or country = "Uruguay" group by country').show()

+-------+-----------------------+
|country|monto_Promedio_Proyecto|
+-------+-----------------------+
|Bolivia|                  3.7E7|
|Uruguay|                 1.22E8|
+-------+-----------------------+



<b>Ejercicio 4<b><p></p>
A partir del dataframe projects_by_country encontrar los 5 paises con mayores montos promedio

In [23]:
spark.sql('select country, AVG(total_amount/num_projects) as monto_Promedio_Proyecto from projects_by_country group by country order by monto_Promedio_Proyecto DESC ').show(5)

+--------------------+-----------------------+
|             country|monto_Promedio_Proyecto|
+--------------------+-----------------------+
|              Poland|               1.3078E9|
|              Turkey|               3.2525E8|
|            Colombia|    3.166666666666667E8|
|            Ethiopia|               3.1125E8|
|Egypt, Arab Repub...|                2.927E8|
+--------------------+-----------------------+
only showing top 5 rows

